In [339]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [340]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [341]:
session = driver.session(database="neo4j")

In [342]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [343]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [344]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [345]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [346]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [347]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [348]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [349]:
cursor = connection.cursor()

In [350]:
my_neo4j_wipe_out_database()

In [351]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [352]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [353]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [354]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [355]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


In [356]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})
"""

session.run(query)

In [357]:
query = """

CALL gds.louvain.stream('ds_graph')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).name AS station_name, communityId AS community
ORDER BY community

"""

my_neo4j_run_query_pandas(query)

,station_name,community
0,depart 16th Street Mission,7
1,arrive 16th Street Mission,7
2,depart 24th Street Mission,7
3,arrive 24th Street Mission,7
4,blue 16th Street Mission,7
...,...,...
209,orange Milpitas,91
210,green Union City,91
211,orange Union City,91
212,green Warm Springs,91


In [366]:
connection.rollback()

query = """

drop table if exists graphy_features
;

create table graphy_features(
    node varchar(32),
    community numeric(5),
    intermediateCommunity varchar(32)
)
;

"""

cursor.execute(query)

connection.commit()

In [367]:
def my_get_node_list():
    "get a list of nodes in the current graph"
    
    query = "match (n) return n.name as name"
    
    result = session.run(query)
    
    node_list = []
    
    for r in result:
        node_list.append(r["name"])
        
    node_list = sorted(node_list)
    
    return node_list

In [368]:
connection.rollback()

query = """

insert into graphy_features
values
(%s, 0, 0)
;

"""

node_list = my_get_node_list()

for node in node_list:
    cursor.execute(query, (node,))

connection.commit()

In [369]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [370]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from graphy_features
order by node

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,node,community,intermediatecommunity
0,arrive 12th Street,0,0
1,arrive 16th Street Mission,0,0
2,arrive 19th Street,0,0
3,arrive 24th Street Mission,0,0
4,arrive Antioch,0,0
...,...,...,...
209,yellow San Bruno,0,0
210,yellow SFO,0,0
211,yellow South San Francisco,0,0
212,yellow Walnut Creek,0,0


In [371]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

result = session.run(query)

for r in result:
    
    query = "update graphy_features set community = %s, intermediateCommunity = %s where node = %s"
    
    cursor.execute(query, (r["community"], r["intermediate_community"], r["name"]))

connection.commit()

In [372]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from graphy_features
order by node

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

In [373]:
df

,node,community,intermediatecommunity
0,arrive 12th Street,67,"{1,5,67}"
1,arrive 16th Street Mission,7,"{3,7,7}"
2,arrive 19th Street,67,"{5,5,67}"
3,arrive 24th Street Mission,7,"{7,7,7}"
4,arrive Antioch,63,"{9,71,63}"
...,...,...,...
209,yellow San Bruno,85,"{81,85,85}"
210,yellow SFO,85,"{85,85,85}"
211,yellow South San Francisco,85,"{89,89,85}"
212,yellow Walnut Creek,63,"{93,93,63}"


In [374]:
len(df['community'].unique())

11